In [1]:
import os
import pandas as pd
from pathlib import Path
from fredapi import Fred
import sys
sys.path.append('../')
from src.data import fred_data

We set FED key

In [2]:
fred = Fred(os.getenv("API_FED"))
fred

It is time to create a dict of the vars and their codes to be downloaded by the FED API

In [3]:
codes_fed = {
    "crude_oil_wti" : "DCOILWTICO",
    "crude_oil_brent" : "DCOILBRENTEU",
    "gas_henry_hub" : "DHHNGSP",
    "nominal_broad_usdollar" : "DTWEXBGS",
    "yuan_dollar_spot_tc" : "DEXCHUS",
    "usdollar_euro" : "DEXUSEU",
    "treasury3month" : "DTB3",
    "treasury1year" : "DTB1YR",
    "treasury4week" : "DTB4WK",
    "treasury6month" : "DTB6",
    "chn_x" : "XTEXVA01CNM667S",
    "chn_m" : "XTIMVA01CNM667S",
    "ue_x" : "XTEXVA01EZM667S",
    "ue_m" : "XTIMVA01EZM667S",
    "usa_x" : "BOPTEXP",
    "usa_m" : "BOPTIMP"
}

We do the call

In [4]:
data_fred = {}
for k, v in codes_fed.items():
    data_fred[k] = fred.get_series(v)

In [5]:
data_fred["crude_oil_wti"]

1986-01-02    25.56
1986-01-03    26.00
1986-01-06    26.53
1986-01-07    25.85
1986-01-08    25.87
              ...  
2022-11-08    88.80
2022-11-09    85.79
2022-11-10    86.52
2022-11-11    89.14
2022-11-14    85.85
Length: 9618, dtype: float64

For creating the correct data frame we generate a fun that the logic is the same in the next code:

In [6]:
temp = data_fred["crude_oil_wti"].to_frame().reset_index()
temp = (temp.rename(columns={temp.columns[0] : "date",
                    temp.columns[1] : "value"})
            .assign(variable = "crude_oil_wti")
        )
temp

,date,value,variable
0,1986-01-02,25.56,crude_oil_wti
1,1986-01-03,26.00,crude_oil_wti
2,1986-01-06,26.53,crude_oil_wti
3,1986-01-07,25.85,crude_oil_wti
4,1986-01-08,25.87,crude_oil_wti
...,...,...,...
9613,2022-11-08,88.80,crude_oil_wti
9614,2022-11-09,85.79,crude_oil_wti
9615,2022-11-10,86.52,crude_oil_wti
9616,2022-11-11,89.14,crude_oil_wti


Here, we apply the fun

In [7]:
data_fred_clean = {}
for k, v in codes_fed.items():
    data_fred_clean[k] = fred_data.cleaning_fred(ds=data_fred[k], var_name=k)

In [8]:
data_fred_clean['gas_henry_hub']

,date,value,var
0,1997-01-07,3.82,gas_henry_hub
1,1997-01-08,3.80,gas_henry_hub
2,1997-01-09,3.61,gas_henry_hub
3,1997-01-10,3.92,gas_henry_hub
4,1997-01-13,4.00,gas_henry_hub
...,...,...,...
6746,2022-11-09,3.46,gas_henry_hub
6747,2022-11-10,4.80,gas_henry_hub
6748,2022-11-11,4.80,gas_henry_hub
6749,2022-11-14,6.24,gas_henry_hub


In [9]:
data_fred_clean["usa_x"]

,date,value,var
0,1992-01-01,50251.0,usa_x
1,1992-02-01,51682.0,usa_x
2,1992-03-01,50294.0,usa_x
3,1992-04-01,50302.0,usa_x
4,1992-05-01,50044.0,usa_x
...,...,...,...
364,2022-05-01,254532.0,usa_x
365,2022-06-01,258763.0,usa_x
366,2022-07-01,259585.0,usa_x
367,2022-08-01,260793.0,usa_x


Concat the dataframes

In [10]:
raw_fred = pd.concat(data_fred_clean, axis=0, ignore_index=True)
raw_fred

,date,value,var
0,1986-01-02,25.56,crude_oil_wti
1,1986-01-03,26.00,crude_oil_wti
2,1986-01-06,26.53,crude_oil_wti
3,1986-01-07,25.85,crude_oil_wti
4,1986-01-08,25.87,crude_oil_wti
...,...,...,...
106187,2022-05-01,340385.00,usa_m
106188,2022-06-01,339642.00,usa_m
106189,2022-07-01,330040.00,usa_m
106190,2022-08-01,326472.00,usa_m


Saving the dataframe

In [11]:
fred_path = Path.cwd().parent.joinpath("data", "raw_fred.csv")
print(f"output path is:{fred_path}")
raw_fred.to_csv(fred_path, index=False)

output path is:c:\Documentos\proyectos\xm4ecast\data\raw_fred.csv
